In [1]:
!pip install wandb

     |████████████████████████████████| 1.3MB 9.9MB/s 
     |████████████████████████████████| 235kB 55.2MB/s 
     |████████████████████████████████| 92kB 32.4MB/s 
     |████████████████████████████████| 460kB 43.9MB/s 
     |████████████████████████████████| 102kB 39.9MB/s 
     |████████████████████████████████| 81kB 31.1MB/s 
     |████████████████████████████████| 256kB 58.8MB/s 
     |████████████████████████████████| 71kB 34.4MB/s 
     |████████████████████████████████| 184kB 59.3MB/s 
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=5b7445bb9e68673c25f6fe6bd2f9e832197dfa3d99134eeae5d1dfd784d50ab3
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Created wheel for watchdog: filename=watchdog-0.9.0-cp36-none-any.whl size=73652 sha256=271feb773c1f7b9dcc2b4ad8ff410932502e0e957ab1febea48a2f9ecc890dbc
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b7

In [ ]:
!mkdir data
!unzip normal_1k.zip
!mv normal data/

#!unzip data/negative.zip 


In [1]:
import argparse
from argparse import RawTextHelpFormatter
import numpy as np
import os

# keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, UpSampling2D, Conv2DTranspose, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import Adam


from absl import app

# logging
import wandb
from datetime import datetime

Using TensorFlow backend.


In [2]:
def create_model(input_dim, latent_dim, verbose=False, save_graph=False, conv=True,
                 adversarial=True):
    '''
    Creates model
    :param input_dim: tuple, dmensions of an image (w*h*ch). W and H has to give modulo of division by 8 = 0
    :param latent_dim: int, number of latent dimensions
    :param verbose: bool, chatty
    :param save_graph: bool, saves latent representation. Work only for 2d latent
    :param conv: bool, make convolutional model
    :param adversarial: bool, make adversarial model
    :return: autoencoder, (discriminator), (generator), encoder, decoder
    '''
    assert input_dim[0]%8 == 0, "Dimension error: Chose H and W dimensions that can be divided by 8 without remnant"
    autoencoder_input = Input(shape=input_dim)
    generator_input = Input(shape=input_dim)
    reshape_dim = int(input_dim[0] / (2 ** 3))
    if conv:
        # Assemble convolutional model
        encoder = Sequential()
        encoder.add(Conv2D(256, kernel_size=(5, 5), input_shape=input_dim, padding='same', activation='relu',
                           data_format="channels_last"))
        encoder.add(MaxPooling2D(pool_size=(2, 2)))
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        encoder.add(MaxPooling2D(pool_size=(2, 2)))
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
        encoder.add(MaxPooling2D(pool_size=(2, 2)))
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Flatten())
        encoder.add(Dense(reshape_dim**2*16, activation='relu'))  # different, was 256
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Dense(reshape_dim**2*16, activation='relu'))  # different, didn't exist
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Dense(latent_dim, activation=None))  # different (was reshaping to 64D)
        
        decoder = Sequential()
        decoder.add(Dense(reshape_dim**2*16, input_shape=(latent_dim,), activation='relu'))
        decoder.add(Dense(reshape_dim**2*16, activation='relu'))
        decoder.add(Dense(reshape_dim**2*32, activation='relu'))
        decoder.add(Reshape((reshape_dim, reshape_dim, 32)))
        decoder.add(UpSampling2D((2, 2)))
        decoder.add(Conv2DTranspose(32, kernel_size=(3, 3), padding='same', activation='relu'))
        decoder.add(UpSampling2D((2, 2)))
        decoder.add(Conv2DTranspose(32, kernel_size=(3, 3), padding='same', activation='relu'))
        decoder.add(UpSampling2D((2, 2)))
        decoder.add(Conv2DTranspose(3, kernel_size=(5, 5), padding='same', activation='sigmoid')) # Relu in CellCognition
        if adversarial:
            discriminator = Sequential()
            discriminator.add(Dense(reshape_dim**2*16, input_shape=(latent_dim,), activation='relu'))
            discriminator.add(Dense(reshape_dim**2*16, activation='relu'))
            discriminator.add(Dense(1, activation='sigmoid'))
    else:
        encoder = Sequential()
        encoder.add(Dense(reshape_dim**2*16, input_shape=input_dim, activation='relu'))
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Dense(reshape_dim**2*16, activation='relu'))
        encoder.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=0.001))
        encoder.add(Dense(latent_dim, activation=None))
        decoder = Sequential()
        decoder.add(Dense(reshape_dim**2*16, input_shape=(latent_dim,), activation='relu'))
        decoder.add(Dense(reshape_dim**2*16, activation='relu'))
        decoder.add(Reshape((reshape_dim, reshape_dim, 3)))
        decoder.add(Dense(input_dim, activation='sigmoid'))
        if adversarial:
            discriminator = Sequential()
            discriminator.add(Dense(1600, input_shape=(latent_dim,), activation='relu'))
            discriminator.add(Dense(1000, activation='relu'))
            discriminator.add(Dense(1, activation='sigmoid'))
    autoencoder = Model(autoencoder_input, decoder(encoder(autoencoder_input)))
    autoencoder.compile(optimizer=Adam(lr=1e-4), loss="mean_squared_error", metrics=['accuracy'])
    if adversarial:
        discriminator.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'])
        discriminator.trainable = False
        generator = Model(generator_input, discriminator(encoder(generator_input)))
        generator.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'])
    if verbose:
        print("Autoencoder Architecture")
        print(autoencoder.summary())
        if adversarial:
            print("Discriminator Architecture")
            print(discriminator.summary())
            print("Generator Architecture")
            print(generator.summary())
    if save_graph:
        plot_model(autoencoder, to_file="autoencoder_graph.png")
        if adversarial:
            plot_model(discriminator, to_file="discriminator_graph.png")
            plot_model(generator, to_file="generator_graph.png")
    if adversarial:
        return autoencoder, discriminator, generator, encoder, decoder
    else:
        return autoencoder, None, None, encoder, decoder

In [3]:

def train(train_data, out, latent_dim, n_epochs, autoencoder, discriminator, generator, encoder, decoder,
          adversarial = True):
    '''
    Function to train autoencoder. Arguments will be taken from argparse
    :param train_data: input data from flow_from_directory
    :param out: dir to save the models
    :param latent_dim: number of latent dimensions
    :param n_epochs: Number of epochs
    :param autoencoder: created autoencoder model
    :param discriminator: created discriminator model
    :param generator: created generator model
    :param encoder: created encoder part of autoencoder
    :param decoder: created decoder part of autoencoder
    :param adversarial: make adversarial model
    :return: trained encoder, decoder, discriminator and generator
    '''
    for epoch in np.arange(1, n_epochs + 1):
        autoencoder_losses = []
        if adversarial:
            discriminator_losses = []
            generator_losses = []
        autoencoder_history = autoencoder.fit_generator(train_data, epochs=1)
        if adversarial:
            batch_index = 0
            discriminator_batch_losses = []
            generator_batch_losses = []
            while batch_index <= train_data.batch_index:
                data = train_data.next()
                data_list = data[0]
                data_size = len(data_list)
                fake_latent = encoder.predict(data_list)
                discriminator_input = np.concatenate((fake_latent, np.random.randn(data_size, latent_dim) * 5.))
                discriminator_labels = np.concatenate((np.zeros((data_size, 1)), np.ones((data_size, 1))))
                discriminator_history = discriminator.fit(x=discriminator_input, y=discriminator_labels, epochs=1,
                                                          batch_size=data_size, validation_split=0.0, verbose=0)
                generator_history = generator.fit(data_list, y=np.ones((data_size, 1)), epochs=1,
                                                  batch_size=data_size, validation_split=0.0, verbose=0)
                batch_index = batch_index + 1
                discriminator_batch_losses.append(discriminator_history.history["loss"])
                generator_batch_losses.append(generator_history.history["loss"])
        autoencoder_losses.append(autoencoder_history.history["loss"])
        # WandB logging
        if adversarial:
            discriminator_losses.append(np.mean(discriminator_batch_losses))
            generator_losses.append(np.mean(generator_batch_losses))
            print("generator_loss = {}\n"
                  "generator_acc = {}".format(
                generator_history.history["loss"],
                generator_history.history["acc"]
            ))
            # WandB logging
            wandb.log({"phase": epoch,
                       "ae_train_loss": autoencoder_history.history["loss"],
                       "ae_train_acc": autoencoder_history.history["acc"],
                       "gen_train_loss": generator_history.history["loss"],
                       "gen_train_acc": generator_history.history["acc"]}, step=epoch)
        else:
            wandb.log({"phase": epoch,
                       "ae_train_loss": autoencoder_history.history["loss"],
                       "ae_train_acc": autoencoder_history.history["acc"]}, step=epoch)
        if epoch % 50 == 0:
            print("\nSaving models...")
            encoder.save(os.path.join(out, 'encoder.h5'))
            decoder.save(os.path.join(out, 'decoder.h5'))
            if adversarial:
                discriminator.save(os.path.join(out, 'discriminator.h5'))
                generator.save(os.path.join(out, 'generator.h5'))
    encoder.save(os.path.join(out, 'encoder.h5'))
    decoder.save(os.path.join(out, 'decoder.h5'))
    if adversarial:
        discriminator.save(os.path.join(out, 'discriminator.h5'))
        generator.save(os.path.join(out, 'generator.h5'))


In [4]:
img_wd = "/home/aogorodnikov/data_aae"
batch = 56
input_dim = (104,104,3)
latent_dim = 32
epoch = 10
conv=True
adversarial=True
out="~/aae/"
# initialize monitoring with WandB
wandb.init()





W&B Run: https://app.wandb.ai/gvaihir_ucsf/image_sudoku/runs/7r97q6xz

In [5]:
autoencoder, discriminator, generator, encoder, decoder = create_model(input_dim=input_dim,
                                                                           latent_dim=latent_dim,
                                                                           verbose=True, save_graph=False,
                                                                           conv=conv,
                                                                           adversarial=adversarial)

Instructions for updating:
Colocations handled automatically by placer.
Autoencoder Architecture
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 104, 104, 3)       0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 32)                22069760  
_________________________________________________________________
sequential_2 (Sequential)    (None, 104, 104, 3)       22053091  
Total params: 44,122,851
Trainable params: 44,111,843
Non-trainable params: 11,008
_________________________________________________________________
None
Discriminator Architecture
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 2704)              89232     
_______________________

/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [6]:
# LOAD DATA
data_loader = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_data = data_loader.flow_from_directory(
    img_wd,
    target_size=(input_dim[0], input_dim[0]),
    batch_size=batch,
    class_mode='input')

Found 1083 images belonging to 1 classes.


In [ ]:
# training mode
train(train_data=train_data, out=out,
    latent_dim=latent_dim, n_epochs=epoch,
    autoencoder=autoencoder, discriminator=discriminator,
    generator=generator, encoder=encoder, decoder=decoder,
    adversarial=adversarial)


Instructions for updating:
Use tf.cast instead.
Epoch 1/1


/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


20/20 [==============================] - 29s 1s/step - loss: 946.0207 - acc: 0.5277
generator_loss = [2.8725650310516357]
generator_acc = [0.05263157933950424]
Epoch 1/1
20/20 [==============================] - 25s 1s/step - loss: 929.9670 - acc: 0.5463
generator_loss = [3.6901795864105225]
generator_acc = [0.05263157933950424]


wandb: Wandb version 0.8.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1


/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 4/20 [=====>........................] - ETA: 24s - loss: 954.8622 - acc: 0.5529

In [27]:
wandb.init(config={"hyper": "parameter"})
autoencoder_history = autoencoder.fit_generator(train_data, epochs=50, callbacks=[WandbCallback()])

Epoch 1/50


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


20/20 [==============================] - 4s 193ms/step - loss: 931.7775 - acc: 0.5512
Epoch 2/50
20/20 [==============================] - 4s 188ms/step - loss: 927.2318 - acc: 0.5524
Epoch 3/50
20/20 [==============================] - 4s 188ms/step - loss: 928.3882 - acc: 0.5489
Epoch 4/50
20/20 [==============================] - 4s 190ms/step - loss: 931.7396 - acc: 0.5512
Epoch 5/50
20/20 [==============================] - 4s 190ms/step - loss: 934.5702 - acc: 0.5494
Epoch 6/50
20/20 [==============================] - 4s 193ms/step - loss: 928.8546 - acc: 0.5516
Epoch 7/50
20/20 [==============================] - 4s 189ms/step - loss: 927.5276 - acc: 0.5517
Epoch 8/50
20/20 [==============================] - 4s 190ms/step - loss: 934.0972 - acc: 0.5487
Epoch 9/50
20/20 [==============================] - 4s 189ms/step - loss: 928.8567 - acc: 0.5498
Epoch 10/50
20/20 [==============================] - 4s 191ms/step - loss: 934.8931 - acc: 0.5468
Epoch 11/50
20/20 [=====================

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
print("generator_loss = {}\n"
                  "generator_acc = {}".format(
                generator_history.history["loss"],
                generator_history.history["acc"]
            ))

In [ ]:
generator_history = generator.fit(data_list, y=np.ones((data_size, 1)), epochs=1,
                                                  batch_size=data_size, validation_split=0.0, verbose=0)

In [ ]:
!python -c 'import keras; print(keras.__version__)'


Using TensorFlow backend.
2.2.5


In [ ]:
def draw(samples):
    fig = plt.figure(figsize=(5 * len(samples), 5))
    gs = gridspec.GridSpec(1, len(samples))
    for i, sample in enumerate(samples):
        ax = plt.Subplot(fig, gs[i])
        ax.imshow((sample["image"] * 255.).reshape(28, 28), cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal')
        ax.set_title(sample["title"])
        fig.add_subplot(ax)
    plt.show(block=False)

In [51]:
choice = np.random.choice(np.arange(9))
original = data_list[choice].reshape(3,104,104)
normalize = colors.Normalize(0., 255.)
original = normalize(original)
latent = encoder.predict(original)
reconstruction = decoder.predict(latent)
draw([{"title": "Original", "image": original}, {"title": "Reconstruction", "image": reconstruction}])

ValueError: ignored

In [31]:
data = train_data.next()
data_list = data[0]

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [50]:
np.max(data_list[2].reshape(3,104,104)[0])

125.31762